In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
alsfrs_orig_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/본 실험/ALSFRS_original.csv')
fvc_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/본 실험/fvc.csv')
weight_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/본 실험/weight.csv')

In [3]:
alsfrs_orig = alsfrs_orig_raw.sort_values(by=['SubjectID','feature_delta'], axis=0).reset_index().drop(columns='index')
fvc = fvc_raw.sort_values(by=['SubjectID', 'feature_delta'], axis=0).reset_index().drop(columns='index')
weight = weight_raw.sort_values(by=['SubjectID', 'feature_delta'], axis=0).reset_index().drop(columns='index')

### Creating target object

 01. Subtracting [FVC < 50]

In [4]:
fvc_a = fvc.groupby('SubjectID').agg(['first', 'last'])
fvc_a = fvc_a.reset_index()
fvc_a

SubjectID fvc_percent      feature_delta     
                     first last         first last
0           89          75   75             0    0
1          329          98   93             0  274
2          406          83   83             0    0
3          411          79   79             0    0
4          533          77   70             0   12
...        ...         ...  ...           ...  ...
7307    999482          79   33             0  434
7308    999823         103   94             0  184
7309    999863          40   36             0  177
7310    999880          53   49             0  266
7311    999990          83   74             0  126

[7312 rows x 5 columns]

In [5]:
# Remove rows with 'last feature_delta == first feature_delta'
fvc_a = fvc_a[fvc_a[('feature_delta', 'last')]!= fvc_a[('feature_delta', 'first')]]
# Select rows with 'first fvc_percent < 50'
fvc_a = fvc_a[fvc_a[('fvc_percent', 'first')]<50]
fvc_a = fvc_a.reset_index().drop(columns='index')
fvc_a

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


SubjectID fvc_percent      feature_delta     
                    first last         first last
0        5291          20    6             0   47
1        6673          21    0             1   73
2       12890          30   12             0  273
3       13147          26   13             0   97
4       16459          28   13             0  130
..        ...         ...  ...           ...  ...
231    981656          15   42             0  180
232    983115          46   60             0  388
233    988844          40   25             0   85
234    991654          36   14             0  182
235    999863          40   36             0  177

[236 rows x 5 columns]

In [6]:
fvc_extent_to_remove = list(fvc_a['SubjectID'])

2. Subtracting [first ALSFRS-Bulbar < 8.0]

In [7]:
alsfrs_B = alsfrs_orig[['SubjectID', 'feature_delta', 'mouth']].reindex(columns=['SubjectID', 'mouth', 'feature_delta'])
alsfrs_B_a = alsfrs_B.groupby('SubjectID').agg(['first'])
alsfrs_B_b = alsfrs_B_a[alsfrs_B_a[('mouth','first')]>8].reset_index()
alsfrs_B_b

,SubjectID,mouth,feature_delta
,,first,first
0,329,11.0,0
1,348,12.0,0
2,708,9.0,0
3,902,11.0,7
4,1234,9.0,0
...,...,...,...
5205,999166,10.0,18
5206,999482,10.0,0
5207,999823,9.0,9


In [8]:
alsfrs_filtered_extent = list(alsfrs_B_b['SubjectID'])

3. Calculating "time to [ALSFRS-B <= 8]"

In [9]:
# Subtract [FVC < 50] & [first ALSFRS-Bulbar < 8.0]
alsfrs_B.query("SubjectID == {0}".format(alsfrs_filtered_extent), inplace=True)
alsfrs_B = alsfrs_B[~alsfrs_B['SubjectID'].isin(fvc_extent_to_remove)]

In [10]:
# Find the first time of [mouth <= 8]
Bulbar_event = alsfrs_B[alsfrs_B['mouth']<=8].groupby('SubjectID').agg(['first']).reset_index()
Bulbar_event

,SubjectID,mouth,feature_delta
,,first,first
0,708,8.0,35
1,1234,8.0,11
2,1736,7.0,48
3,3085,8.0,118
4,5067,5.0,98
...,...,...,...
1870,998756,8.0,35
1871,998894,8.0,191
1872,999166,8.0,319


In [11]:
# Coding [Bulbar<=8] event as '1'
Bulbar_event_1 = Bulbar_event.iloc[:, [0,-1]]
Bulbar_event_1.columns = ['SubjectID', 'time_B']

Bulbar_event_1_sublist = list(Bulbar_event['SubjectID'])

Bulbar_event_1['status_B'] = 1
print("There are",len(Bulbar_event_1['SubjectID'].unique()), "subjects whose status_B = 1")

There are 1875 subjects whose status_B = 1


In [12]:
# Otherwise '0'
Bulbar_event_0 = alsfrs_B[~alsfrs_B['SubjectID'].isin(Bulbar_event_1_sublist)]
Bulbar_event_0 = Bulbar_event_0.groupby('SubjectID').agg(['last']).reset_index().drop(columns=('mouth', 'last'))
Bulbar_event_0.columns = ['SubjectID', 'time_B']
Bulbar_event_0['status_B'] = 0
print("There are",len(Bulbar_event_0['SubjectID'].unique()), "subjects whose status_B = 0")

There are 3263 subjects whose status_B = 0


In [13]:
alsfrs_B_coded = pd.concat([Bulbar_event_1, Bulbar_event_0]).sort_values(by='SubjectID', axis=0)
alsfrs_B_coded

,SubjectID,time_B,status_B
0,329,274,0
1,348,401,0
0,708,35,1
2,902,154,0
1,1234,11,1
...,...,...,...
1872,999166,319,1
1873,999482,80,1
1874,999823,91,1
3261,999880,266,0


4. Calculating "time to [weight loss >=10%]"

In [14]:
len(weight['SubjectID'].unique())

7588

In [15]:
# Subtract [FVC < 50] & [first ALSFRS-Bulbar < 8.0]
weight.query("SubjectID == {0}".format(alsfrs_filtered_extent), inplace=True)
weight = weight[~weight['SubjectID'].isin(fvc_extent_to_remove)]

In [16]:
weight_a = weight.groupby('SubjectID').agg(['first'])
weight_a.columns = ['initial_weight', 'feature_delta']
D = weight_a.to_dict()
weight_dict = D['initial_weight']

In [17]:
weight_IDs = list(weight_a.reset_index()['SubjectID'])

In [18]:
weight_dict.keys()

dict_keys([329, 348, 708, 902, 1234, 1309, 1736, 2492, 2540, 2598, 2956, 3085, 3809, 3971, 3981, 4390, 4752, 4772, 4918, 5067, 5155, 5588, 5658, 5773, 5918, 5936, 6747, 6786, 7399, 7468, 7750, 7795, 7799, 7824, 7835, 7858, 8150, 8227, 8258, 8480, 8586, 8708, 8709, 9100, 9469, 9494, 9641, 9821, 10059, 10136, 10244, 10253, 10456, 10517, 10584, 10956, 10995, 11052, 11081, 11684, 11800, 11813, 11941, 12118, 12485, 12826, 12856, 13105, 13165, 13212, 13503, 13613, 14096, 14465, 14836, 14840, 15472, 15511, 16030, 16483, 16497, 16656, 16782, 17035, 17090, 17179, 17390, 17603, 17703, 17756, 17760, 17864, 17937, 17959, 18216, 18304, 18341, 18424, 18470, 18747, 19174, 19225, 19246, 19256, 19344, 19574, 19613, 19764, 20121, 20417, 20626, 20801, 20912, 21508, 22671, 22674, 22741, 22904, 23315, 23364, 23479, 23509, 23719, 23939, 23973, 24198, 24428, 24570, 24571, 24755, 25214, 25280, 25745, 25800, 25941, 26066, 26082, 26114, 26279, 26540, 26556, 26577, 26673, 26814, 26830, 27214, 27463, 27577, 27616

In [19]:
df = pd.DataFrame([])
for k in weight_dict.keys():
  value = weight_dict[k]*0.9
  df2 = weight[(weight['SubjectID']== k) & (weight['weight']<= value)]
  df = pd.concat([df, df2])
df

,SubjectID,weight,feature_delta
4,329,45.8,72.0
5,329,41.1,106.0
6,329,43.4,135.0
7,329,42.6,168.0
8,329,42.7,189.0
...,...,...,...
50126,997276,56.8,70.0
50243,999482,74.0,252.0
50244,999482,75.0,317.0
50245,999482,74.0,374.0


In [20]:
df = df.groupby('SubjectID').agg(['first']).reset_index()
# Coding [weight loss >=10%] event as '1'
weight_loss_1 = df.drop(columns='weight')
weight_loss_1.columns = ['SubjectID', 'time_W']
weight_loss_1['status_W'] = 1

weight_loss_1_sublist = list(weight_loss_1['SubjectID'])
print("There are",len(weight_loss_1['SubjectID'].unique()), "subjects whose status_W = 1")

There are 1269 subjects whose status_W = 1


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [21]:
# Otherwise '0'
weight_loss_0 = weight[~weight['SubjectID'].isin(weight_loss_1_sublist)]
weight_loss_0 = weight_loss_0.groupby('SubjectID').agg(['last']).reset_index().drop(columns=('weight', 'last'))
weight_loss_0.columns = ['SubjectID', 'time_W']
weight_loss_0['status_W'] = 0
print("There are",len(Bulbar_event_0['SubjectID'].unique()), "subjects whose status_W = 0")

There are 3263 subjects whose status_W = 0


In [22]:
weight_loss_coded = pd.concat([weight_loss_1, weight_loss_0]).sort_values(by='SubjectID', axis=0)
weight_loss_coded

,SubjectID,time_W,status_W
0,329,72.0,1
0,348,401.0,0
1,708,91.0,1
1,902,154.0,0
2,1234,560.0,0
...,...,...,...
3584,999166,390.0,0
1268,999482,252.0,1
3585,999823,184.0,0
3586,999880,266.0,0


5. Finalizing target object "time to gas, status"

In [23]:
Optimal_Gas = pd.merge(alsfrs_B_coded, weight_loss_coded, on='SubjectID', how='outer')
Optimal_Gas

,SubjectID,time_B,status_B,time_W,status_W
0,329,274,0,72.0,1.0
1,348,401,0,401.0,0.0
2,708,35,1,91.0,1.0
3,902,154,0,154.0,0.0
4,1234,11,1,560.0,0.0
...,...,...,...,...,...
5133,999166,319,1,390.0,0.0
5134,999482,80,1,252.0,1.0
5135,999823,91,1,184.0,0.0
5136,999880,266,0,266.0,0.0


In [24]:
df_3 = list([])
df_4 = list([])
for i in range(0, 5138):
  if (Optimal_Gas.iloc[i]['status_B']!=1) and (Optimal_Gas.iloc[i]['status_W']==1):
    df_3.append(Optimal_Gas.iloc[i]['time_W'])
    df_4.append(1)
  elif (Optimal_Gas.iloc[i]['status_B']==1) and (Optimal_Gas.iloc[i]['status_W']!=1):
    df_3.append(Optimal_Gas.iloc[i]['time_B'])
    df_4.append(1)
  elif (Optimal_Gas.iloc[i]['status_B']==1) and (Optimal_Gas.iloc[i]['status_W']==1):
    df_3.append(min([Optimal_Gas.iloc[i]['time_B'], Optimal_Gas.iloc[i]['time_W']]))
    df_4.append(1)
  else:
    df_3.append(max([Optimal_Gas.iloc[i]['time_B'], Optimal_Gas.iloc[i]['time_W']]))
    df_4.append(0)

In [25]:
Optimal_Gas['time_to_gas'] = df_3
Optimal_Gas['status'] = df_4
Optimal_Gas

,SubjectID,time_B,status_B,time_W,status_W,time_to_gas,status
0,329,274,0,72.0,1.0,72.0,1
1,348,401,0,401.0,0.0,401.0,0
2,708,35,1,91.0,1.0,35.0,1
3,902,154,0,154.0,0.0,154.0,0
4,1234,11,1,560.0,0.0,11.0,1
...,...,...,...,...,...,...,...
5133,999166,319,1,390.0,0.0,319.0,1
5134,999482,80,1,252.0,1.0,80.0,1
5135,999823,91,1,184.0,0.0,91.0,1
5136,999880,266,0,266.0,0.0,266.0,0


### Preprocessing Features

1. ALSFRS_Q

In [26]:
extent = list(Optimal_Gas['SubjectID'])

In [27]:
alsfrs_raw = alsfrs_orig.copy()

In [28]:
alsfrs_raw.query("SubjectID == {0}".format(extent), inplace=True)
alsfrs_raw.query("feature_delta < 92", inplace=True)
alsfrs_raw.drop(columns=['Q5a_Cutting_without_Gastrostomy', 'Q5b_Cutting_with_Gastrostomy'], inplace=True)

In [29]:
alsfrs_raw['Bulbar'] = alsfrs_raw['mouth'] - alsfrs_raw['Q2_Salivation']
alsfrs_raw['motor'] = alsfrs_raw['hands'] + alsfrs_raw['trunk'] + alsfrs_raw['leg']

In [30]:
alsfrs_raw

,SubjectID,feature_delta,ALSFRS_Total,Q1_Speech,Q2_Salivation,Q3_Swallowing,Q4_Handwriting,Q5_Cutting,Q6_Dressing_and_Hygiene,Q7_Turning_in_Bed,Q8_Walking,Q9_Climbing_Stairs,Q10_Respiratory,mouth,hands,leg,trunk,respiratory,Bulbar,motor
0,329,0,27.0,4.0,3.0,4.0,3.0,2.0,3.0,2.0,2.0,1.0,3.0,11.0,5.0,3.0,5.0,3.0,8.0,13.0
1,329,8,26.5,4.0,3.0,4.0,3.0,1.5,3.0,2.0,2.0,1.0,3.0,11.0,4.5,3.0,5.0,3.0,8.0,12.5
2,329,16,26.0,4.0,3.0,4.0,3.0,1.0,3.0,2.0,2.0,1.0,3.0,11.0,4.0,3.0,5.0,3.0,8.0,12.0
3,329,42,30.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,2.0,1.0,4.0,11.0,6.0,3.0,6.0,4.0,8.0,15.0
4,329,72,30.0,4.0,3.0,4.0,3.0,2.0,3.0,4.0,2.0,2.0,3.0,11.0,5.0,4.0,7.0,3.0,8.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59246,999880,49,24.0,4.0,4.0,4.0,2.0,1.0,1.0,2.0,2.0,0.0,4.0,12.0,3.0,2.0,3.0,4.0,8.0,8.0
59247,999880,70,25.0,4.0,4.0,4.0,2.0,1.0,1.0,3.0,2.0,0.0,4.0,12.0,3.0,2.0,4.0,4.0,8.0,9.0
59255,999990,12,22.0,3.0,4.0,4.0,2.0,1.0,1.0,2.0,1.0,0.0,4.0,11.0,3.0,1.0,3.0,4.0,7.0,7.0
59256,999990,40,19.0,3.0,4.0,4.0,0.0,1.0,1.0,1.0,1.0,0.0,4.0,11.0,1.0,1.0,2.0,4.0,7.0,4.0


In [31]:
df = alsfrs_raw.groupby('SubjectID').agg('mean')
df.drop(columns=['feature_delta','mouth', 'hands', 'leg', 'trunk', 'respiratory'], inplace=True)
df = df.reset_index()

In [32]:
df.columns=['SubjectID', 'mean_alsfrs_total', 'mean_Q1_Speech', 'mean_Q2_Salivation', 'mean_Q3_Swallowing', 'mean_Q4_Handwriting', 'mean_Q5_Cutting', 'mean_Q6_Dressing_and_Hygiene', 'mean_Q7_Turning_in_Bed', 'mean_Q8_Walking', 'mean_Q9_Climbing_Stairs', 'mean_Q10_Respiratory', 'mean_Bulbar', 'mean_motor']

In [33]:
mean_alsfrs_scores = df.copy()
mean_alsfrs_scores = mean_alsfrs_scores.sort_values(by=['SubjectID'], axis=0, ascending=True)
mean_alsfrs_scores

,SubjectID,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor
0,329,27.90,4.00,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.00,1.200000,3.200000,8.000000,13.700000
1,348,30.25,4.00,4.000000,4.000000,3.500000,2.00,2.250000,3.500000,2.00,1.000000,4.000000,8.000000,14.250000
2,708,32.25,3.00,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.00,2.833333,2.833333,5.666667,20.916667
3,902,32.25,4.00,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.50,1.000000,3.000000,7.250000,18.250000
4,1234,21.25,2.00,3.000000,4.000000,0.000000,1.00,1.000000,2.500000,2.75,1.000000,4.000000,6.000000,8.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,999166,35.00,3.00,3.000000,4.000000,4.000000,4.00,4.000000,3.000000,3.00,3.000000,4.000000,7.000000,21.000000
5132,999482,36.25,2.75,2.750000,3.750000,4.000000,3.00,4.000000,4.000000,4.00,4.000000,4.000000,6.500000,23.000000
5133,999823,35.25,2.75,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.00,4.000000,4.000000,5.500000,22.750000
5134,999880,24.20,4.00,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.00,0.000000,4.000000,8.000000,8.200000


In [34]:
df3 = alsfrs_raw.groupby('SubjectID').agg(['first', 'last'])
df3['interval'] = df3.iloc[:, 1] - df3.iloc[:, 0]
df3['slope_alsfrs_total'] = (df3.iloc[:,3] - df3.iloc[:,2])/df3['interval']
df3['slope_Q1_Speech'] = (df3.iloc[:,5] - df3.iloc[:,4])/df3['interval']
df3['slope_Q2_Salivation'] = (df3.iloc[:,7] - df3.iloc[:,6])/df3['interval']
df3['slope_Q3_Swallowing'] = (df3.iloc[:,9] - df3.iloc[:,8])/df3['interval']
df3['slope_Q4_Handwriting'] = (df3.iloc[:,11] - df3.iloc[:,10])/df3['interval']
df3['slope_Q5_Cutting'] = (df3.iloc[:,13] - df3.iloc[:,12])/df3['interval']
df3['slope_Q6_Dressing_and_Hygiene'] = (df3.iloc[:,15] - df3.iloc[:,14])/df3['interval']
df3['slope_Q7_Turning_in_Bed'] = (df3.iloc[:,17] - df3.iloc[:,15])/df3['interval']
df3['slope_Q8_Walking'] = (df3.iloc[:,19] - df3.iloc[:,18])/df3['interval']
df3['slope_Q9_Climbing_Stairs'] = (df3.iloc[:,21] - df3.iloc[:,20])/df3['interval']
df3['slope_Q10_Respiratory'] = (df3.iloc[:,23] - df3.iloc[:,22])/df3['interval']
df3['slope_Bulbar'] = (df3.iloc[:,35] - df3.iloc[:,34])/df3['interval']
df3['slope_motor'] = (df3.iloc[:,37] - df3.iloc[:,36])/df3['interval']

In [35]:
df4 = df3.iloc[:, 39:]
df4 = df4.reset_index()

In [36]:
slope_alsfrs_scores = df4.copy()
slope_alsfrs_scores = slope_alsfrs_scores.sort_values(by=['SubjectID'], axis=0, ascending=True)
slope_alsfrs_scores

,SubjectID,slope_alsfrs_total,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor
,,,,,,,,,,,,,,
0,329,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667
1,348,-0.016393,0.000000,0.000000,0.000000,0.016393,0.0,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393
2,708,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989
3,902,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714
4,1234,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,-0.014286,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,999166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN
5132,999482,-0.037500,-0.012500,-0.012500,-0.012500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.025000,0.000000
5133,999823,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000


In [37]:
alsfrs_tot_slope = slope_alsfrs_scores.sort_values(by=['SubjectID'], axis=0, ascending=True)
alsfrs_tot_slope = alsfrs_tot_slope.reset_index()

In [38]:
alsfrs_tot_slope.iloc[:,1:]

,SubjectID,slope_alsfrs_total,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor
,,,,,,,,,,,,,,
0,329,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667
1,348,-0.016393,0.000000,0.000000,0.000000,0.016393,0.0,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393
2,708,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989
3,902,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714
4,1234,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,-0.014286,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,999166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN
5132,999482,-0.037500,-0.012500,-0.012500,-0.012500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.025000,0.000000
5133,999823,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000


In [39]:
df1 = mean_alsfrs_scores
df2 = alsfrs_tot_slope.iloc[:,2:]

alsfrs_q = pd.concat([df1, df2], axis=1)
alsfrs_q.columns = ['SubjectID','mean_alsfrs_total','mean_Q1_Speech','mean_Q2_Salivation','mean_Q3_Swallowing','mean_Q4_Handwriting','mean_Q5_Cutting','mean_Q6_Dressing_and_Hygiene',
'mean_Q7_Turning_in_Bed','mean_Q8_Walking','mean_Q9_Climbing_Stairs','mean_Q10_Respiratory','mean_Bulbar','mean_motor','slope_alsfrs_tot','slope_Q1_Speech',
'slope_Q2_Salivation','slope_Q3_Swallowing','slope_Q4_Handwriting','slope_Q5_Cutting','slope_Q6_Dressing_and_Hygiene','slope_Q7_Turning_in_Bed','slope_Q8_Walking','slope_Q9_Climbing_Stairs','slope_Q10_Respiratory',
'slope_Bulbar','slope_motor']
alsfrs_q

,SubjectID,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_alsfrs_tot,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor
0,329,27.90,4.00,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.00,1.200000,3.200000,8.000000,13.700000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667
1,348,30.25,4.00,4.000000,4.000000,3.500000,2.00,2.250000,3.500000,2.00,1.000000,4.000000,8.000000,14.250000,-0.016393,0.000000,0.000000,0.000000,0.016393,0.0,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393
2,708,32.25,3.00,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.00,2.833333,2.833333,5.666667,20.916667,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989
3,902,32.25,4.00,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.50,1.000000,3.000000,7.250000,18.250000,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714
4,1234,21.25,2.00,3.000000,4.000000,0.000000,1.00,1.000000,2.500000,2.75,1.000000,4.000000,6.000000,8.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,-0.014286,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,999166,35.00,3.00,3.000000,4.000000,4.000000,4.00,4.000000,3.000000,3.00,3.000000,4.000000,7.000000,21.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN
5132,999482,36.25,2.75,2.750000,3.750000,4.000000,3.00,4.000000,4.000000,4.00,4.000000,4.000000,6.500000,23.000000,-0.037500,-0.012500,-0.012500,-0.012500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.025000,0.000000
5133,999823,35.25,2.75,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.00,4.000000,4.000000,5.500000,22.750000,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000
5134,999880,24.20,4.00,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.00,0.000000,4.000000,8.000000,8.200000,0.014286,0.000000,0.000000,0.000000,0.000000,0.0,0.014286,0.028571,0.000000,0.000000,0.000000,0.000000,0.014286


2. Static features & Time resolved features

In [40]:
demographics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/본 실험/demographic.csv')
als_hx = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/본 실험/als_hx.csv')
fvc_3mo = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/본 실험/fvc_3mo_meta.csv')

In [41]:
demographics = demographics[['SubjectID', 'Age', 'Gender']]
als_hx['diag_minus_onset'] = als_hx['diag_delta'] - als_hx['onset_delta']

In [42]:
static = pd.DataFrame(columns=['SubjectID'])
static_list = [demographics, als_hx]
for i in static_list :
    df = i
    static = static.merge(df, on='SubjectID', how='outer')

In [43]:
fvc_3mo = fvc_3mo[['SubjectID','fvc_slope', 'fvc_mean']]

3. Merge features

In [44]:
features_1 = static.merge(fvc_3mo, on='SubjectID', how='outer')
features_1.query("SubjectID == {0}".format(extent), inplace=True)
features_1 = features_1.reset_index().drop(columns='index')

In [45]:
features = features_1.merge(alsfrs_q, on='SubjectID', how='outer')
features

,SubjectID,Age,Gender,diag_delta,onset_delta,onset_site,diag_minus_onset,fvc_slope,fvc_mean,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_alsfrs_tot,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor
0,329,38.0,F,-1068.0,-1706.0,Limb,638.0,0.048171,101.25,27.90,4.00,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.00,1.200000,3.200000,8.000000,13.700000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667
1,348,52.0,F,-302.0,-501.0,Other,199.0,NaN,NaN,30.25,4.00,4.000000,4.000000,3.500000,2.00,2.250000,3.500000,2.00,1.000000,4.000000,8.000000,14.250000,-0.016393,0.000000,0.000000,0.000000,0.016393,0.0,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393
2,708,46.0,M,-986.0,-1070.0,Bulbar,84.0,-0.061928,51.00,32.25,3.00,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.00,2.833333,2.833333,5.666667,20.916667,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989
3,902,77.0,F,-49.0,-514.0,Limb,465.0,-0.041707,89.60,32.25,4.00,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.50,1.000000,3.000000,7.250000,18.250000,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714
4,1234,38.0,M,NaN,NaN,NaN,NaN,0.009325,71.00,21.25,2.00,3.000000,4.000000,0.000000,1.00,1.000000,2.500000,2.75,1.000000,4.000000,6.000000,8.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,-0.014286,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,999166,60.0,M,-49.0,-999.0,Bulbar,950.0,-0.333333,75.00,35.00,3.00,3.000000,4.000000,4.000000,4.00,4.000000,3.000000,3.00,3.000000,4.000000,7.000000,21.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN
5134,999482,48.0,M,NaN,NaN,NaN,NaN,-0.069309,78.75,36.25,2.75,2.750000,3.750000,4.000000,3.00,4.000000,4.000000,4.00,4.000000,4.000000,6.500000,23.000000,-0.037500,-0.012500,-0.012500,-0.012500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.025000,0.000000
5135,999823,55.0,M,-84.0,-492.0,Limb,408.0,-0.025981,100.80,35.25,2.75,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.00,4.000000,4.000000,5.500000,22.750000,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000
5136,999880,51.0,M,-2105.0,-2865.0,Limb,760.0,-0.079112,50.25,24.20,4.00,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.00,0.000000,4.000000,8.000000,8.200000,0.014286,0.000000,0.000000,0.000000,0.000000,0.0,0.014286,0.028571,0.000000,0.000000,0.000000,0.000000,0.014286


In [46]:
Gend = {"F":0, "M":1}
onsetsite = {"Other":0, "Bulbar":1, "Limb":0, "Limb and Bulbar":0}
# Bubar_onset = 1, non-Bulbar_onset = 0

In [47]:
features = features.replace({'Gender':Gend})
features = features.replace({'onset_site':onsetsite})
features

,SubjectID,Age,Gender,diag_delta,onset_delta,onset_site,diag_minus_onset,fvc_slope,fvc_mean,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_alsfrs_tot,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor
0,329,38.0,0,-1068.0,-1706.0,0.0,638.0,0.048171,101.25,27.90,4.00,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.00,1.200000,3.200000,8.000000,13.700000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667
1,348,52.0,0,-302.0,-501.0,0.0,199.0,NaN,NaN,30.25,4.00,4.000000,4.000000,3.500000,2.00,2.250000,3.500000,2.00,1.000000,4.000000,8.000000,14.250000,-0.016393,0.000000,0.000000,0.000000,0.016393,0.0,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393
2,708,46.0,1,-986.0,-1070.0,1.0,84.0,-0.061928,51.00,32.25,3.00,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.00,2.833333,2.833333,5.666667,20.916667,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989
3,902,77.0,0,-49.0,-514.0,0.0,465.0,-0.041707,89.60,32.25,4.00,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.50,1.000000,3.000000,7.250000,18.250000,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714
4,1234,38.0,1,NaN,NaN,NaN,NaN,0.009325,71.00,21.25,2.00,3.000000,4.000000,0.000000,1.00,1.000000,2.500000,2.75,1.000000,4.000000,6.000000,8.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,-0.014286,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,999166,60.0,1,-49.0,-999.0,1.0,950.0,-0.333333,75.00,35.00,3.00,3.000000,4.000000,4.000000,4.00,4.000000,3.000000,3.00,3.000000,4.000000,7.000000,21.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN
5134,999482,48.0,1,NaN,NaN,NaN,NaN,-0.069309,78.75,36.25,2.75,2.750000,3.750000,4.000000,3.00,4.000000,4.000000,4.00,4.000000,4.000000,6.500000,23.000000,-0.037500,-0.012500,-0.012500,-0.012500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.025000,0.000000
5135,999823,55.0,1,-84.0,-492.0,0.0,408.0,-0.025981,100.80,35.25,2.75,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.00,4.000000,4.000000,5.500000,22.750000,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000
5136,999880,51.0,1,-2105.0,-2865.0,0.0,760.0,-0.079112,50.25,24.20,4.00,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.00,0.000000,4.000000,8.000000,8.200000,0.014286,0.000000,0.000000,0.000000,0.000000,0.0,0.014286,0.028571,0.000000,0.000000,0.000000,0.000000,0.014286


In [48]:
Optimal_Gas = Optimal_Gas[['SubjectID', 'time_to_gas', 'status']]
feat_n_targ_combined_with_nans = features.merge(Optimal_Gas, on='SubjectID', how='outer')
feat_n_targ_combined_with_nans

,SubjectID,Age,Gender,diag_delta,onset_delta,onset_site,diag_minus_onset,fvc_slope,fvc_mean,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_alsfrs_tot,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,time_to_gas,status
0,329,38.0,0,-1068.0,-1706.0,0.0,638.0,0.048171,101.25,27.90,4.00,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.00,1.200000,3.200000,8.000000,13.700000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667,72.0,1
1,348,52.0,0,-302.0,-501.0,0.0,199.0,NaN,NaN,30.25,4.00,4.000000,4.000000,3.500000,2.00,2.250000,3.500000,2.00,1.000000,4.000000,8.000000,14.250000,-0.016393,0.000000,0.000000,0.000000,0.016393,0.0,-0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,-0.016393,401.0,0
2,708,46.0,1,-986.0,-1070.0,1.0,84.0,-0.061928,51.00,32.25,3.00,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.00,2.833333,2.833333,5.666667,20.916667,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989,35.0,1
3,902,77.0,0,-49.0,-514.0,0.0,465.0,-0.041707,89.60,32.25,4.00,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.50,1.000000,3.000000,7.250000,18.250000,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714,154.0,0
4,1234,38.0,1,NaN,NaN,NaN,NaN,0.009325,71.00,21.25,2.00,3.000000,4.000000,0.000000,1.00,1.000000,2.500000,2.75,1.000000,4.000000,6.000000,8.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.028571,-0.014286,0.000000,0.000000,0.000000,0.000000,11.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,999166,60.0,1,-49.0,-999.0,1.0,950.0,-0.333333,75.00,35.00,3.00,3.000000,4.000000,4.000000,4.00,4.000000,3.000000,3.00,3.000000,4.000000,7.000000,21.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-inf,NaN,NaN,NaN,NaN,NaN,319.0,1
5134,999482,48.0,1,NaN,NaN,NaN,NaN,-0.069309,78.75,36.25,2.75,2.750000,3.750000,4.000000,3.00,4.000000,4.000000,4.00,4.000000,4.000000,6.500000,23.000000,-0.037500,-0.012500,-0.012500,-0.012500,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.025000,0.000000,80.0,1
5135,999823,55.0,1,-84.0,-492.0,0.0,408.0,-0.025981,100.80,35.25,2.75,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.00,4.000000,4.000000,5.500000,22.750000,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000,91.0,1
5136,999880,51.0,1,-2105.0,-2865.0,0.0,760.0,-0.079112,50.25,24.20,4.00,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.00,0.000000,4.000000,8.000000,8.200000,0.014286,0.000000,0.000000,0.000000,0.000000,0.0,0.014286,0.028571,0.000000,0.000000,0.000000,0.000000,0.014286,266.0,0


4. Remove rows "onset_delta = NaN"

In [49]:
data = feat_n_targ_combined_with_nans.copy()

In [50]:
data = data[['SubjectID', 'onset_delta']]

In [51]:
data2 = data.dropna(subset=['onset_delta'])
extent2 = list(data2["SubjectID"].unique())

In [62]:
feat_n_targ_combined_with_nans = feat_n_targ_combined_with_nans.query("SubjectID == {0}".format(extent2))
feat_n_targ_combined_with_nans = feat_n_targ_combined_with_nans[feat_n_targ_combined_with_nans.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
feat_n_targ_combined_with_nans

,SubjectID,Age,Gender,diag_delta,onset_delta,onset_site,diag_minus_onset,fvc_slope,fvc_mean,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_alsfrs_tot,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,time_to_gas,status
0,329,38.0,0,-1068.0,-1706.0,0.0,638.0,0.048171,101.250000,27.900000,4.000000,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.0,1.200000,3.200000,8.000000,13.700000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667,72.0,1
2,708,46.0,1,-986.0,-1070.0,1.0,84.0,-0.061928,51.000000,32.250000,3.000000,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.0,2.833333,2.833333,5.666667,20.916667,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989,35.0,1
3,902,77.0,0,-49.0,-514.0,0.0,465.0,-0.041707,89.600000,32.250000,4.000000,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.5,1.000000,3.000000,7.250000,18.250000,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714,154.0,0
6,1736,41.0,1,-511.0,-570.0,0.0,59.0,0.000000,71.000000,16.500000,2.500000,3.000000,3.000000,1.500000,1.00,1.000000,0.500000,0.5,0.000000,3.500000,5.500000,4.500000,-0.104167,-0.020833,-0.041667,0.000000,-0.020833,0.0,0.000000,-0.020833,-0.020833,0.000000,0.020833,-0.020833,-0.062500,48.0,1
7,2492,63.0,1,-209.0,-365.0,0.0,156.0,-0.235294,114.000000,30.000000,4.000000,4.000000,4.000000,3.000000,3.00,2.000000,3.000000,3.0,1.000000,3.000000,8.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,401.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5127,998542,70.0,1,-73.0,-287.0,0.0,214.0,0.000000,85.000000,27.500000,4.000000,3.000000,4.000000,4.000000,3.00,2.000000,2.000000,2.0,1.000000,2.500000,8.000000,14.000000,-0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.125000,0.000000,0.000000,21.0,0
5128,998756,64.0,1,-241.0,-612.0,1.0,371.0,-0.255102,73.333333,29.333333,2.666667,2.666667,3.000000,3.000000,3.00,3.000000,4.000000,3.0,3.000000,2.000000,5.666667,19.000000,-0.017544,0.000000,-0.017544,0.000000,0.000000,0.0,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,35.0,1
5135,999823,55.0,1,-84.0,-492.0,0.0,408.0,-0.025981,100.800000,35.250000,2.750000,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.0,4.000000,4.000000,5.500000,22.750000,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000,91.0,1
5136,999880,51.0,1,-2105.0,-2865.0,0.0,760.0,-0.079112,50.250000,24.200000,4.000000,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.0,0.000000,4.000000,8.000000,8.200000,0.014286,0.000000,0.000000,0.000000,0.000000,0.0,0.014286,0.028571,0.000000,0.000000,0.000000,0.000000,0.014286,266.0,0


### Data imputation

In [59]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model

In [63]:
X = feat_n_targ_combined_with_nans

In [64]:
columns = X.columns
imputer = IterativeImputer(sample_posterior = True)
ar = imputer.fit_transform(X)
X_imputed = pd.DataFrame(ar, columns = columns)

In [65]:
X_imputed

,SubjectID,Age,Gender,diag_delta,onset_delta,onset_site,diag_minus_onset,fvc_slope,fvc_mean,mean_alsfrs_total,mean_Q1_Speech,mean_Q2_Salivation,mean_Q3_Swallowing,mean_Q4_Handwriting,mean_Q5_Cutting,mean_Q6_Dressing_and_Hygiene,mean_Q7_Turning_in_Bed,mean_Q8_Walking,mean_Q9_Climbing_Stairs,mean_Q10_Respiratory,mean_Bulbar,mean_motor,slope_alsfrs_tot,slope_Q1_Speech,slope_Q2_Salivation,slope_Q3_Swallowing,slope_Q4_Handwriting,slope_Q5_Cutting,slope_Q6_Dressing_and_Hygiene,slope_Q7_Turning_in_Bed,slope_Q8_Walking,slope_Q9_Climbing_Stairs,slope_Q10_Respiratory,slope_Bulbar,slope_motor,time_to_gas,status
0,329.0,38.0,0.0,-1068.0,-1706.0,0.0,638.0,0.048171,101.250000,27.900000,4.000000,3.000000,4.000000,3.000000,1.90,3.000000,2.600000,2.0,1.200000,3.200000,8.000000,13.700000,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.041667,72.0,1.0
1,708.0,46.0,1.0,-986.0,-1070.0,1.0,84.0,-0.061928,51.000000,32.250000,3.000000,2.833333,2.666667,3.166667,3.75,3.166667,4.000000,4.0,2.833333,2.833333,5.666667,20.916667,-0.021978,0.000000,-0.010989,-0.010989,0.000000,0.0,0.010989,0.000000,0.000000,0.000000,-0.010989,-0.010989,0.010989,35.0,1.0
2,902.0,77.0,0.0,-49.0,-514.0,0.0,465.0,-0.041707,89.600000,32.250000,4.000000,3.750000,3.250000,4.000000,4.00,2.750000,4.000000,2.5,1.000000,3.000000,7.250000,18.250000,-0.035714,0.000000,0.000000,0.011905,0.000000,0.0,-0.023810,0.023810,-0.011905,0.000000,-0.011905,0.011905,-0.035714,154.0,0.0
3,1736.0,41.0,1.0,-511.0,-570.0,0.0,59.0,0.000000,71.000000,16.500000,2.500000,3.000000,3.000000,1.500000,1.00,1.000000,0.500000,0.5,0.000000,3.500000,5.500000,4.500000,-0.104167,-0.020833,-0.041667,0.000000,-0.020833,0.0,0.000000,-0.020833,-0.020833,0.000000,0.020833,-0.020833,-0.062500,48.0,1.0
4,2492.0,63.0,1.0,-209.0,-365.0,0.0,156.0,-0.235294,114.000000,30.000000,4.000000,4.000000,4.000000,3.000000,3.00,2.000000,3.000000,3.0,1.000000,3.000000,8.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,401.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,998542.0,70.0,1.0,-73.0,-287.0,0.0,214.0,0.000000,85.000000,27.500000,4.000000,3.000000,4.000000,4.000000,3.00,2.000000,2.000000,2.0,1.000000,2.500000,8.000000,14.000000,-0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.125000,0.000000,0.000000,21.0,0.0
2324,998756.0,64.0,1.0,-241.0,-612.0,1.0,371.0,-0.255102,73.333333,29.333333,2.666667,2.666667,3.000000,3.000000,3.00,3.000000,4.000000,3.0,3.000000,2.000000,5.666667,19.000000,-0.017544,0.000000,-0.017544,0.000000,0.000000,0.0,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,35.0,1.0
2325,999823.0,55.0,1.0,-84.0,-492.0,0.0,408.0,-0.025981,100.800000,35.250000,2.750000,3.000000,2.750000,4.000000,3.75,3.000000,4.000000,4.0,4.000000,4.000000,5.500000,22.750000,-0.024390,-0.012195,0.000000,-0.012195,0.000000,0.0,0.000000,0.012195,0.000000,0.000000,0.000000,-0.024390,0.000000,91.0,1.0
2326,999880.0,51.0,1.0,-2105.0,-2865.0,0.0,760.0,-0.079112,50.250000,24.200000,4.000000,4.000000,4.000000,2.000000,1.00,0.400000,2.800000,2.0,0.000000,4.000000,8.000000,8.200000,0.014286,0.000000,0.000000,0.000000,0.000000,0.0,0.014286,0.028571,0.000000,0.000000,0.000000,0.000000,0.014286,266.0,0.0
